# Semester 1 Project Submission



## Elliot

### The Movie Database Data Cleanse

In [7]:
#import needed libraries
import pandas as pd

In [8]:
#read tmdb file, make sure values are in the same format, and drop null values
tmdb_movie_db = pd.read_csv('databases/tmdb.movies.csv', na_filter=True, na_values='[]', encoding = 'utf-8', index_col = 0)
tmdb_movie_db.dropna()
#drop extra title column
tmdb_movie_db = tmdb_movie_db.drop('original_title', axis = 1)
#standardize release date into date format
tmdb_movie_db['release_date'] = pd.to_datetime(tmdb_movie_db['release_date'])
#strip whitespace from title
tmdb_movie_db['title'] = tmdb_movie_db['title'].str.strip()
#remove duplicates
tmdb_movie_db = tmdb_movie_db.drop_duplicates()
#convert genre_ids to name
#df.where()

tmdb_movie_db

,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,0.600,2018-06-22,Trailer Made,0.0,1


In [9]:
#information regarding The Movie Database
tmdb_movie_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25497 entries, 0 to 26516
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   genre_ids          23036 non-null  object        
 1   id                 25497 non-null  int64         
 2   original_language  25497 non-null  object        
 3   popularity         25497 non-null  float64       
 4   release_date       25497 non-null  datetime64[ns]
 5   title              25497 non-null  object        
 6   vote_average       25497 non-null  float64       
 7   vote_count         25497 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 1.8+ MB


In [10]:
#check to see if there any null values
tmdb_movie_db.isnull().sum()

genre_ids            2461
id                      0
original_language       0
popularity              0
release_date            0
title                   0
vote_average            0
vote_count              0
dtype: int64

In [11]:
#check to see that there are no duplicates
tmdb_movie_db.duplicated().value_counts() 

False    25497
dtype: int64

In [12]:
#view shape of data
tmdb_movie_db.shape

(25497, 8)

### The Numbers Data Cleanse

In [45]:
#read tn file, make sure values are in the same format, and drop null values
tn_movie_db = pd.read_csv('databases/tn.movie_budgets.csv', na_filter=True, na_values='[]', encoding = 'utf-8', index_col = 0)
tn_movie_db.dropna()
#standardize release date into date format
tn_movie_db['release_date'] = pd.to_datetime(tn_movie_db['release_date'])
#strip whitespace from title
tn_movie_db['movie'] = tn_movie_db['movie'].str.strip()
#strip $
tn_movie_db['production_budget'] = tn_movie_db['production_budget'].str.strip('$')
tn_movie_db['domestic_gross'] = tn_movie_db['domestic_gross'].str.strip('$')
tn_movie_db['worldwide_gross'] = tn_movie_db['worldwide_gross'].str.strip('$')
#remove ,
tn_movie_db['production_budget'] = tn_movie_db['production_budget'].replace(",", "", regex=True)
tn_movie_db['domestic_gross'] = tn_movie_db['domestic_gross'].replace(",", "", regex=True)
tn_movie_db['worldwide_gross'] = tn_movie_db['worldwide_gross'].replace(",", "", regex=True)
#convert to integers
tn_movie_db['production_budget'] = tn_movie_db['production_budget'].astype(int)
tn_movie_db['domestic_gross'] = tn_movie_db['domestic_gross'].astype(int)
tn_movie_db['worldwide_gross'] = tn_movie_db['worldwide_gross'].astype(float)
#create metric ratios
tn_movie_db['dom_gross / budget'] = tn_movie_db['domestic_gross'] / tn_movie_db['production_budget']
tn_movie_db['ww_gross / budget'] = tn_movie_db['worldwide_gross'] / tn_movie_db['production_budget']

tn_movie_db

,release_date,movie,production_budget,domestic_gross,worldwide_gross,dom_gross / budget,ww_gross / budget
id,,,,,,,
1,2009-12-18,Avatar,425000000,760507625,2.776345e+09,1.789430,6.532577
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1.045664e+09,0.587101,2.546673
3,2019-06-07,Dark Phoenix,350000000,42762350,1.497624e+08,0.122178,0.427892
4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1.403014e+09,1.388403,4.243841
5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1.316722e+09,1.956408,4.153696
...,...,...,...,...,...,...,...
78,2018-12-31,Red 11,7000,0,0.000000e+00,0.000000,0.000000
79,1999-04-02,Following,6000,48482,2.404950e+05,8.080333,40.082500
80,2005-07-13,Return to the Land of Wonders,5000,1338,1.338000e+03,0.267600,0.267600


In [14]:
#information regarding The Numbers
tn_movie_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   release_date        5782 non-null   datetime64[ns]
 1   movie               5782 non-null   object        
 2   production_budget   5782 non-null   int32         
 3   domestic_gross      5782 non-null   int32         
 4   worldwide_gross     5782 non-null   float64       
 5   ww_gross / budget   5782 non-null   float64       
 6   dom_gross / budget  5782 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int32(2), object(1)
memory usage: 316.2+ KB


In [15]:
#check to see if there any null values
tn_movie_db.isnull().sum()

release_date          0
movie                 0
production_budget     0
domestic_gross        0
worldwide_gross       0
ww_gross / budget     0
dom_gross / budget    0
dtype: int64

In [16]:
#check to see that there are no duplicates
tn_movie_db.duplicated().value_counts() 

False    5782
dtype: int64

In [17]:
#view shape of data
tn_movie_db.shape

(5782, 7)

### Data Visualization

Merge both tables to combine data to connect genres to production costs and boxoffice revenue

In [37]:
#merge title and movie name
tn_movie_db = tn_movie_db.rename(columns = {"movie":"title"})

#special cases where movie titles do not match
tn_movie_db['title'].replace({'Harry Potter and the Deathly Hallows: Part I' : 'Harry Potter and the Deathly Hallows: Part 1'}, inplace=True)
tn_movie_db['title'].replace({'Harry Potter and the Deathly Hallows: Part II' : 'Harry Potter and the Deathly Hallows: Part 2'}, inplace=True)
tmdb_movie_db['title'].replace({'Fast & Furious 6':'Fast and Furious 6'}, inplace=True)
tn_movie_db['title'].replace({'Star Wars: The Force Awakens' : 'Star Wars Ep. VII: The Force Awakens'}, inplace=True)
tn_movie_db['title'].replace({'Star Wars: The Last Jedi' : 'Star Wars Ep. VIII: The Last Jedi'}, inplace=True)

#merge databases
movie_db = tmdb_movie_db.merge(tn_movie_db, how = 'left')

#drop null values
movie_db.dropna()

,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count,production_budget,domestic_gross,worldwide_gross,dom_gross / budget,ww_gross / budget
0,"[12, 14, 10751]",12444,en,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,125000000.0,296131568.0,960431568.0,2.369053,7.683453
1,"[14, 12, 16, 10751]",10191,en,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,165000000.0,217581232.0,494870992.0,1.318674,2.999218
2,"[12, 28, 878]",10138,en,28.515,2010-05-07,Iron Man 2,6.8,12368,170000000.0,312433331.0,621156389.0,1.837843,3.653861
3,"[16, 35, 10751]",862,en,28.005,1995-11-22,Toy Story,7.9,10174,30000000.0,191796233.0,364545516.0,6.393208,12.151517
4,"[28, 878, 12]",27205,en,27.920,2010-07-16,Inception,8.3,22186,160000000.0,292576195.0,835524642.0,1.828601,5.222029
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23711,"[53, 18, 27]",510284,en,5.972,2019-02-01,Braid,6.4,18,1660000.0,0.0,80745.0,0.000000,0.048642
23747,"[18, 10752]",514407,en,5.599,2018-10-26,Indivisible,5.4,13,2700000.0,3511417.0,3588305.0,1.300525,1.329002
23795,"[18, 28, 80]",547590,en,5.274,2019-05-03,El Chicano,9.0,1,8000000.0,700261.0,700261.0,0.087533,0.087533
23907,"[18, 35, 28, 80]",506971,ur,4.486,2018-07-20,Teefa in Trouble,7.6,11,1500000.0,0.0,98806.0,0.000000,0.065871


In [ ]:

#table = pd.concat([tmdb_movie_db, tn_movie_db], axis = 1)
#tmdb_movie_db.join(tn_movie_db, how = 'outer')
#tn_movie_db.merge(tmdb_movie_db, how = 'left')

#tmdb_movie_db.iloc[0].title
#tn_movie_db

#df.loc[df['column_name'].isin(some_values)]

In [40]:
# Import matplotlib
import matplotlib.pyplot as plt

# Set plot space as inline for inline plots and qt for external plots
%matplotlib inline

In [47]:
#information regarding genres
movie_db['genre_ids'].value_counts()

[99]                    3565
[18]                    2119
[35]                    1622
[27]                    1125
[53]                     466
                        ... 
[28, 16, 10770]            1
[12, 14, 10749, 878]       1
[14, 35, 878, 10751]       1
[12, 10749, 18]            1
[10751, 12, 28]            1
Name: genre_ids, Length: 2476, dtype: int64

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

ax.bar(movie_db['dom_gross / budget'].mean(), 10, tick_label = movie_db['genre_ids'])

ax.set_title('Average Dom_gross / Budget Ratio per Genres')
ax.set_ylabel('Average Dom_gross / Budget')
ax.set_xlabel('Movie Genres');



#x = movie_db['genre_ids']

#y = movie_db['dom_gross / budget'].mean()

#plt.hist(x, bins = 1000, edgecolor='black', color='#00C8AD')
#plt.xlabel('Movie Genres')
#plt.ylabel('Average Dom_gross / Budget')
#plt.title('Average Dom_gross / Budget Ratio per Genres')
#plt.show()

In [ ]:
x = movie_db['genre_ids']

y = movie_db['ww_gross / budget'].mean()

plt.hist(x, bins = 50, edgecolor='black')
plt.xlabel('Movie Genres')
plt.ylabel('Average WW_gross / Budget')
plt.title('Average WW_gross / Budget Ratio per Genres')
plt.show()

In [ ]:
(profit per genre)

## Ian

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Frida

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Enrique

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## John

In [ ]:
# Your code here - remember to use markdown cells for comments as well!